# eLiquid Mixing Calculator

Calculates quantities in both millilitres and grams of nic-less flavour, flavourless nic, raw PG and raw VG to add based on desired target levels.

## Usage

This calculation assumes you have:
* One or more zero nicotine flavourings, and know their PG/VG ratios
* Flavourless eliquid with nicotine, and know the nicotine mg/100ml value and the PG/VG ratio
* Pure PG
* Pure VG

## Input Section

In [8]:
# Specify all values as floats

# List of zero nicotine flavouring components.
# Specify strength multiplier, eg. singler 1, doubler 2, tripler 3.
# if it's a concentrate, specify required %age of concentrate as a whole number (eg 15%, specify 15)
zero_nic_flavours = {
    "name": "Mint Chocolate",
    "amount_ratio_unit": "v", # "m" indicates the amount ratio is by mass, "v" indicates it is by volume
    "components": [
        { "name": "Mint Sliced",
          "strength": 3.0,
          "pg_ratio": 0.3, # by volume (ml)
          "vg_ratio": 0.7, # bu volume (ml)
          "amount_ratio": 0.5, # The ratio of this liquid in the final zero-nic flavouring mix
        },
        { "name": "Chocolate Indulgence",
          "strength": 1.0,
          "pg_ratio": 0.5,
          "vg_ratio": 0.5,
          "amount_ratio": 0.5,
        }
    ]
}

# Base nicotine flvourless liquid, strength unit is mg/100ml
nic_base_strength = 100.0
nic_base_pg_ratio = 0.8 # by volume (ml)
nic_base_vg_ratio = 0.2 # by volume (ml)

# Final required amount to create in ml
amount_to_create = 50.0
# Final required nic level in mg/100ml
target_nic_level = 5.0
# Final desired target pv/vg ratio. These may not be achievable given the above inputs and will be adjusted automatically if required.
desired_target_pg_ratio = 0.2 # by volume (ml)
desired_target_vg_ratio = 0.8 # by volume (ml)

# Number of decimal places to round final results to
round_to = 2

# log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
log_level = "DEBUG"

## Code

In [9]:
import logging

logging.basicConfig()
logging.basicConfig(level=logging.NOTSET)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)

# Constants
pg_grams_per_ml = 1.04
vg_grams_per_ml = 1.26

# input validation
#if (flavour_pg_ratio + flavour_vg_ratio) != 1:
#    raise Exception("Flavoured zero-nicotine liquid pg/vg ratio must add to 1")
#if (nic_base_pg_ratio + nic_base_vg_ratio) != 1:
#    raise Exception("Base nicotine flvourless liquid pg/vg ratio must add to 1")
#if (desired_target_pg_ratio + desired_target_vg_ratio) != 1:
#    raise Exception("Desired target pg/vg ratio must add to 1")


    
# Calculate flavour and nic solution amounts
#
# derive final strenth and pg/vg ratio of flavour mix

flavour_pg_ratio = 0
flavour_vg_ratio = 0
flavour_strength = 0

amount_ratio_unit = zero_nic_flavours['amount_ratio_unit']
components = zero_nic_flavours['components']
for component in components:
    flavour_pg_ratio += component['pg_ratio'] * component['amount_ratio'] / (amount_ratio_unit == "m" and pg_grams_per_ml or 1)
    flavour_vg_ratio += component['vg_ratio'] * component['amount_ratio'] / (amount_ratio_unit == "m" and vg_grams_per_ml or 1)
    flavour_strength += component['strength'] * component['amount_ratio']

amount_nic_base_ml = amount_to_create * target_nic_level / nic_base_strength
amount_flavour_ml = amount_to_create / flavour_strength


# Calculate how much pg or vg to add to the flavour+nic solution to achieve the desired pg/vg ratio
#
# get volumes of existing pg/vg components after mixing flavoured and nic base eliquids
# these will be the minimum amounts of pg and vg in the final yield
min_pg_ml = flavour_pg_ratio * amount_flavour_ml + nic_base_pg_ratio * amount_nic_base_ml
min_vg_ml = flavour_vg_ratio * amount_flavour_ml + nic_base_vg_ratio * amount_nic_base_ml
min_total_ml = min_pg_ml + min_vg_ml

# calculate min/max pg/vg ratio thresholds
max_pg_ml = amount_to_create - min_total_ml + min_pg_ml
max_vg_ml = amount_to_create - min_total_ml + min_vg_ml
min_pg_ratio = min_pg_ml / amount_to_create
max_pg_ratio = max_pg_ml / amount_to_create
min_vg_ratio = min_vg_ml / amount_to_create
max_vg_ratio = max_vg_ml / amount_to_create

# adjust target pg/vg ratios to land within min/max thresholds if required
target_pg_ratio = desired_target_pg_ratio
target_vg_ratio = desired_target_vg_ratio
if (min_pg_ratio <= target_pg_ratio <= max_pg_ratio and min_vg_ratio <= target_vg_ratio <= max_vg_ratio):
    ratio_adjusted = False
else:
    ratio_adjusted = True
    if target_pg_ratio < min_pg_ratio:
        target_pg_ratio = min_pg_ratio
        target_vg_ratio = 1 - target_pg_ratio
    if target_pg_ratio > max_pg_ratio:
        target_pg_ratio = max_pg_ratio
        target_vg_ratio = 1 - target_pg_ratio
    if target_vg_ratio < min_vg_ratio:
        target_vg_ratio = min_vg_ratio
        target_pg_ratio = 1 - target_vg_ratio
    if target_vg_ratio > max_vg_ratio:
        target_vg_ratio = max_vg_ratio
        target_pg_ratio = 1 - target_vg_ratio

logger.debug(f"min_pg_ml:    {min_pg_ml}")
logger.debug(f"min_vg_ml:    {min_vg_ml}")
logger.debug(f"min_total_ml: {min_total_ml}")

# calculate amount of pg or vg to add to the solution to achieve the target pg/vg ratio
adjusted_volume = min_pg_ml / target_pg_ratio
logger.debug(f"adjusted_volume (fixed PG): {adjusted_volume}")
logger.debug(f"adjusted_volume (fixed VG): {min_vg_ml / target_vg_ratio}")
if adjusted_volume < min_total_ml:
    logger.debug("need to bring PG up...")
    adjusted_volume = min_vg_ml / target_vg_ratio
    amount_of_vg_to_add = 0
    amount_of_pg_to_add = adjusted_volume - min_total_ml    
elif adjusted_volume > min_total_ml:
    logger.debug("need to bring VG up...")
    amount_of_vg_to_add = adjusted_volume - min_total_ml
    amount_of_pg_to_add = 0
else:
    logger.debug("VG/PG ratio already correct...")
    amount_of_vg_to_add = 0
    amount_of_pg_to_add = 0
    
# calculate how much pg and/or vg to add to the now adjusted solution to achieve the desired yield at the target pg/vg ratio
top_up_ml = amount_to_create - (amount_of_vg_to_add + amount_of_pg_to_add + min_total_ml)
logger.debug(f"added {amount_of_pg_to_add} ml pg")
logger.debug(f"added {amount_of_vg_to_add} ml vg")
logger.debug(f"top_up_ml:      {top_up_ml}")
    
total_top_up_pg_ml = amount_of_pg_to_add + top_up_ml * target_pg_ratio
total_top_up_vg_ml = amount_of_vg_to_add + top_up_ml * target_vg_ratio

# if the initial flavourless concentration is too weak, we may arrive at a negative number for top_up_ml in order to maintain the desired
# final flavour concentrate with the desired nicotine level. In this case we sacrifice flavour by diluting it in order to arrive at an exact
# nicotine level.

flavour_diluted_factor = None
raw_amount_flavour_ml = amount_flavour_ml
if total_top_up_pg_ml < 0:
    amount_flavour_ml = amount_flavour_ml + total_top_up_pg_ml
    total_top_up_pg_ml = 0
if total_top_up_vg_ml < 0:
    amount_flavour_ml = amount_flavour_ml + total_top_up_vg_ml
    total_top_up_vg_ml = 0
if raw_amount_flavour_ml > amount_flavour_ml:
    flavour_diluted_factor = amount_flavour_ml / raw_amount_flavour_ml * 100
    flavour_diluted_factor = f"{flavour_diluted_factor:.{round_to}f}%"
    

# calculate final yield
final_yield_ml = amount_flavour_ml + amount_nic_base_ml + total_top_up_pg_ml + total_top_up_vg_ml
logger.debug(f"final_yield:   {final_yield_ml}")
    
# calculate mass
amount_nic_base_gm = amount_nic_base_ml * nic_base_pg_ratio * pg_grams_per_ml + amount_nic_base_ml * nic_base_vg_ratio * vg_grams_per_ml
amount_flavour_gm = amount_flavour_ml * flavour_pg_ratio * pg_grams_per_ml + amount_flavour_ml * flavour_vg_ratio * vg_grams_per_ml
total_top_up_pg_gm = total_top_up_pg_ml * pg_grams_per_ml
total_top_up_vg_gm = total_top_up_vg_ml * vg_grams_per_ml
total_mass = amount_nic_base_gm + amount_flavour_gm + total_top_up_pg_gm + total_top_up_vg_gm

# final calculations for presenting yield values
final_pg_ratio = 100 * (min_pg_ml + total_top_up_pg_ml) / final_yield_ml
final_vg_ratio = 100 * (min_vg_ml + total_top_up_vg_ml) / final_yield_ml
final_nic_ratio = amount_nic_base_ml * nic_base_strength / final_yield_ml

# print all results
print(f"""
Inputs:
    
    Flavour mix (strength, PG/VG ratio):          {flavour_strength}x, {flavour_pg_ratio}/{flavour_vg_ratio}
    Flavourless nicotine (strength, PG/VG ratio): {nic_base_strength} mg/100ml, {nic_base_pg_ratio}/{nic_base_vg_ratio}
    Target final nicotine level:                  {target_nic_level} mg/100ml
    Desired target PG/VG ratio:                   {desired_target_pg_ratio}/{desired_target_vg_ratio} ({ratio_adjusted and "not " or ""}achievable)
    
Achievable Thresholds:

    Max possible PG/VG ratio:     {max_pg_ratio:.{round_to}f}/{min_vg_ratio:.{round_to}f}
    Min possible PG/VG ratio:     {min_pg_ratio:.{round_to}f}/{max_vg_ratio:.{round_to}f}
    Target PG/VG Ratio:           {target_pg_ratio:.{round_to}f}/{target_vg_ratio:.{round_to}f} {ratio_adjusted and "(adjusted)" or ""}

Final Mix amounts:

    Flavour to add:              {amount_flavour_ml:.{round_to}f} ml ({amount_flavour_gm:.{round_to}f} g)
    Flavourless nicotine to add: {amount_nic_base_ml:.{round_to}f} ml ({amount_nic_base_gm:.{round_to}f} g)
    PG to add:                   {total_top_up_pg_ml:.{round_to}f} ml ({total_top_up_pg_gm:.{round_to}f} g)
    VG to add:                   {total_top_up_vg_ml:.{round_to}f} ml ({total_top_up_vg_gm:.{round_to}f} g)

Final Yield:

    {final_yield_ml:.{round_to}f} ml ({total_mass:.{round_to}f}g) @ {final_pg_ratio:.{round_to}f}/{final_vg_ratio:.{round_to}f} PG/VG ratio, {final_nic_ratio:.{round_to}f} mg/100ml nicotine
    
    {flavour_diluted_factor and "NOTE: Final flavour concentration has been diluted by " + flavour_diluted_factor + " to maintain correct nicotine ratio" or ""}
""")

DEBUG:__main__:min_pg_ml:    12.0
DEBUG:__main__:min_vg_ml:    15.5
DEBUG:__main__:min_total_ml: 27.5
DEBUG:__main__:adjusted_volume (fixed PG): 50.0
DEBUG:__main__:adjusted_volume (fixed VG): 20.394736842105264
DEBUG:__main__:need to bring VG up...
DEBUG:__main__:added 0 ml pg
DEBUG:__main__:added 22.5 ml vg
DEBUG:__main__:top_up_ml:      0.0
DEBUG:__main__:final_yield:   50.0



Inputs:
    
    Flavour mix (strength, PG/VG ratio):          2.0x, 0.4/0.6
    Flavourless nicotine (strength, PG/VG ratio): 100.0 mg/100ml, 0.8/0.2
    Target final nicotine level:                  5.0 mg/100ml
    Desired target PG/VG ratio:                   0.2/0.8 (not achievable)
    
Achievable Thresholds:

    Max possible PG/VG ratio:     0.69/0.31
    Min possible PG/VG ratio:     0.24/0.76
    Target PG/VG Ratio:           0.24/0.76 (adjusted)

Final Mix amounts:

    Flavour to add:              25.00 ml (29.30 g)
    Flavourless nicotine to add: 2.50 ml (2.71 g)
    PG to add:                   0.00 ml (0.00 g)
    VG to add:                   22.50 ml (28.35 g)

Final Yield:

    50.00 ml (60.36g) @ 24.00/76.00 PG/VG ratio, 5.00 mg/100ml nicotine
    
    

